# CRYSTALS Dilithium

O CRYSTALS Dilithium trata-se de uma técnica de assinatura digital pós-quântica baseado no M-LWE.

## Parâmetros:
Neste esquema de assinaturas $n$, $q$ e $h$ são fixos:
- $n = 256$
- $q = 2^{23}-2^{13} +1 = 8380417$
- $h = 60$
- $r = 1753$, sendo $r$ uma raiz primitiva de ordem $n$ de $-1$

Os restantes parâmetros são os recomendados, ou seja

- $K = 4$
- $l = 3$
- $\gamma = 523776$
- $\alpha = 261888$
- $\eta = 6$
- $\beta = 325$

## KeyGen:
- Gera-se $A \in \mathcal{R}_{q}^{k \times l}$
- Gera-se $s_1$ pertencente a $S_{\eta}^{l}$ e $s_2$ pertencente a $S_{\eta}^k$ , sendo $S_{\eta}$ o conjunto de todos os polinómios tal que a
norma infinita de qualquer polinómio é menos que $\eta$
- Seja $t = A \cdot s1 + s2$
- A chave pública é $(A,t)$
- A chave privada é $(s_1,s_2)$

## Sign:
- Gera-se $y$ pertencente a $S_{\gamma-1}^{l}$
- $w = HighBits(Ay,2\alpha)$
- $c = H(m || w)$
- $z = y + cs_1$
- Enquanto a norma infinita de $z$ for maior ou igual que $\gamma - \beta$ ou a norma infinita de $LowBits(Ay - cs_2,2\alpha)$ for maior ou igual do que $\alpha - \beta$, voltamos ao início

A assinatura é o par $(z,c)$.

## Verify:
- $w' = HighBits(Az - ct,2\alpha)$
- Se a norma infinita de $z$ for maior ou igual que $\gamma - \beta$ ou $c \neq H(m || w')$ rejeita-se a assinatura, caso contrário a assinatura é aceite.

In [8]:
import hashlib


class Dilithium:

    def __init__(self):
        self.n = 256
        self.q = 8380417
        self.h = 60
        self.r = 1753
        self.k = 4
        self.l = 3
        self.gama = 523776
        self.alfa = 261888
        self.eta = 6
        self.beta = 325


    #FUNCOES AUXILIARES:


    # Geracao do conjunto S
    def gen_S(self,lim,tam):
        Zq.<z> = PolynomialRing(GF(self.q))
        Rq.<z> = Zq.quotient(z^self.n+1)
        S = []
        for i in range(tam):
            pol = []
            for j in range(self.n):
                pol.append(randint(1,lim))
            S.append(Rq(pol))
        S = matrix(Rq,tam,1,S)
        return S

    def Decompose(self,c,t):
        Rq = IntegerModRing(self.q)
        Rt = IntegerModRing(t)
        r = int(Rq(c))
        r0 = int(Rt(r))
        if r0 > t/2:
            r0 = r0 - int(t)
        if r - r0 == self.q - 1:
            r1 = 0
            r0 = r0 - 1
        else:
            r1 = (r - r0)/(int(t))
        return (r1,r0)

    def HighBits(self,c):
        x = self.Decompose(c,2*self.alfa)
        return x[0]

    def LowBits(self,c):
        x = self.Decompose(c,2*self.alfa)
        return x[1]

    def HBpol(self,pol):
        k = pol.list()
        for i in range(len(k)):
            h = k[i]
            h = h.list()
            for j in range(len(h)):
                h[j] = self.HighBits(int(h[j]))
            k[i] = h
        return k

    def LBpol(self,pol):
        k = pol.list()
        for i in range(len(k)):
            h = k[i]
            h = h.list()
            for j in range(len(h)):
                h[j] = self.LowBits(int(h[j]))
            k[i] = h
        return k

    # Geracao de um elemento aleatorio em B60
    def SampleInBall(self,r):
        """ 'r' é usado como fonte de aleatoriedade (obs: deve ser suficientemente grande para possibilitar o rejection-sampling dos 60 valores de 'j') """
        sl = "{:064b}".format(int(r[:8].hex(),16)) # primeiros 60 bit são usados para selecção sinal.
        k = 8 # posição para 'rejection-sampling'
        c = [0] * 256 # coeficientes
        #print("primeiros 64bit (para s):",sl)
        for i in range (196,256):
            while (int(r[k])>i):
                #print('R:', int(r[k]), ">", i)
                k +=1 # rejeita valores fora da gama pretendida.
            j = int(r[k])
            k += 1
            s = int(sl[i-196])
            #print("j=", j, "s=", s)
            c[i] = c[j]
            c[j] = (-1)^(s)
        return c

    def Shake(self,a,b, n=int(256)):
        """ default-value para 'n' dá uma folga para 60 rejeições... """
        shake = hashlib.shake_256()
        shake.update(a)
        shake.update(b)
        s = shake.digest(n)
        return s

    def Hash(self,a,b,n=int(256)):
        r = self.Shake(a,b,n)
        c = self.SampleInBall(r)
        return c

    def norma_infinito(self,pol,n):
        J = pol.list()
        for i in range(len(J)):
            k = J[i]
            K = k.list()
            for j in range(len(K)):
                K[j] = abs(int(K[j]))
            J[i] = K
        L = []
        for i in range(len(J)):
            L.append(max(J[i]))
        return max(L)

    def norma_inf_vet(self,vetor):
        Zq.<z> = PolynomialRing(GF(self.q))
        Rq.<z> = Zq.quotient(z^self.n + 1)
        for i in range(vetor.nrows()):
            norm = self.norma_infinito(vetor[i],self.q)
            vetor[i] = norm
        return max(vetor)

    def norma_inf_matriz(self,matriz):
        L = []
        for i in range(len(matriz)):
            k = matriz[i]
            for j in range(len(k)):
                if k[j] < 0:
                    k[j] = abs(k[j])
                L.append(max(k))
        for i in range(len(L)):
            J = []
            J.append(max(L))
        return J[0]



    # FUNCOES PRINCIPAIS


    def Keygen(self):
        Zx.<x> = ZZ[]
        R.<x> = Zx.quotient(x^self.n+1)
        Zq.<z> = PolynomialRing(GF(self.q))
        Rq.<z> = Zq.quotient(z^self.n+1)

        # PASSO 1
        K = []
        for i in range(self.k*self.l):
            K.append(Rq.random_element())
        A = matrix(Rq,self.k,self.l,K)

        # PASSO 2
        s1 = self.gen_S(self.eta,self.l) #s1: matriz 3x1
        s2 = self.gen_S(self.eta,self.k) #s2:matriz 4x1

        # PASSO 3
        t = A*s1 + s2 #t: matriz 4x1

        # PASSO 4 E 5
        self.pubKey = (A,t)
        self.privKey = (s1,s2)
        return(self.pubKey,self.privKey)


    def Sign(self,m):

        A = self.pubKey[0]
        s1 = self.privKey[0]
        s2 = self.privKey[1]

        Zx.<x> = ZZ[]
        R.<x> = Zx.quotient(x^self.n + 1)
        Zq.<z> = PolynomialRing(GF(self.q))
        Rq.<z> = Zq.quotient(z^self.n + 1)

        # PASSO 1
        z = None

        # PASSO 2
        while z == None:

            # PASSO 3
            y = self.gen_S(self.gama - 1,self.l)

            # PASSO 4
            Ay = A*y
            w = self.HBpol(Ay)

            # PASSO 5
            u = str(w).encode()
            k = m.encode()
            c = self.Hash(k,u)
            cq = Rq(c)

            # PASSO 6
            z = matrix(y + cq*s1)

            aa = self.norma_inf_vet(z)[0]
            bb = int(self.gama - self.beta)
            cc = int(self.norma_inf_matriz(self.LBpol(Ay-cq*s2)))
            dd = int(self.alfa - self.beta)

            # PASSO 7 E 8
            if aa >= bb or cc >= dd:
                z = None

        z = matrix(y + cq*s1)
        return (z,c)


    def Verify(self,m,sig):
        Zx.<x> = ZZ[]
        R.<x> = Zx.quotient(x^self.n+1)
        Zq.<z> = PolynomialRing(GF(self.q))
        Rq.<z> = Zq.quotient(z^self.n+1)

        A = self.pubKey[0]
        t = self.pubKey[1]
        z = sig[0]
        c = sig[1]
        cq = Rq(c)

        # PASSO 1
        Az = A*z
        w = self.HBpol(Az - cq*t)

        # PASSO 2 E 3
        u = str(w).encode()
        k = m.encode()
        novo_c = self.Hash(k,u)
        aa = int(self.norma_inf_vet(z)[0])
        bb = int(self.gama - self.beta)
        if aa >= bb or c != novo_c:
            print('Assinatura rejeitada!')
        else:
            print('Assinatura aceite!')

In [9]:
Teste = Dilithium()
chaves = Teste.Keygen()
m = bin(123456789)
sign = Teste.Sign(m)
print('Mensagem: ', m)
#print('Par de chaves: ',chaves)
#print('Assinatura: ',sign)
Teste.Verify(m,sign)

Mensagem:  0b111010110111100110100010101
Assinatura aceite!


In [11]:
for i in range (1,100):
    Teste = Dilithium()
    Teste.Keygen()
    m = bin(randint(1,999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999))
    sign = Teste.Sign(m)
    Teste.Verify(m,sign)

Assinatura aceite!


Assinatura aceite!


Assinatura aceite!


Assinatura aceite!


Assinatura aceite!


...

Assinatura aceite!


Assinatura aceite!


Assinatura aceite!


Assinatura aceite!


Assinatura aceite!


# Zita Abreu